In [ ]:
import os

# Define paths
base_path = '/kaggle/working'
project_name = 'Double-Pendulum-Simulation'
project_root = os.path.join(base_path, project_name)

# Clone or pull repository
if not os.path.exists(project_root):
    os.chdir(base_path)
    !git clone https://github.com/nthday-jpg/Double-Pendulum-Simulation.git
    print("Repository cloned successfully!")
else:
    os.chdir(project_root)
    !git pull
    print("Repository updated successfully!")
%cd {base_path}

# Ensure project root is in PYTHONPATH for script's imports
os.environ['PYTHONPATH'] = project_root


In [ ]:
!python {project_root}/scripts/generate_data.py \
    --output_dir {project_root}/data/raw \
    --num_trajectories 20 \
    --num_points 3000 \
    --t_start 0.0 \
    --t_end 2.0 \
    --check_energy

Generating 20 trajectories...
Deriving equations symbolically (this may take a moment)...
Symbolic derivation complete!
  Trajectory 000 (m1=1.00, m2=1.00, l1=1.00, l2=1.00): Energy drift = 0.000%
  Saved: /kaggle/working/Double-Pendulum-Simulation/data/raw/trajectory_000.npz and /kaggle/working/Double-Pendulum-Simulation/data/raw/parameters_000.json
  Trajectory 001 (m1=1.00, m2=1.00, l1=1.00, l2=1.00): Energy drift = 0.000%
  Saved: /kaggle/working/Double-Pendulum-Simulation/data/raw/trajectory_001.npz and /kaggle/working/Double-Pendulum-Simulation/data/raw/parameters_001.json
  Trajectory 002 (m1=1.00, m2=1.00, l1=1.00, l2=1.00): Energy drift = 0.000%
  Saved: /kaggle/working/Double-Pendulum-Simulation/data/raw/trajectory_002.npz and /kaggle/working/Double-Pendulum-Simulation/data/raw/parameters_002.json
  Trajectory 003 (m1=1.00, m2=1.00, l1=1.00, l2=1.00): Energy drift = 0.000%
  Saved: /kaggle/working/Double-Pendulum-Simulation/data/raw/trajectory_003.npz and /kaggle/working/Doub

In [3]:
# Remove old dataset
!rm -rf /kaggle/working/Double-Pendulum-Simulation/data/raw/

In [ ]:
# Training Configuration
# Experiment
seed = 42
run_name = None  # Auto-generated if None
checkpoint_path = None#"runs/run_20260112_135314/checkpoints/best_model.pth"

# Data
data_dir = f"{project_root}/data/raw"
val_split = 0.2
test_split = 0.1
normalize_time = True
normalize_state = False

# Model Architecture
model = "pinn"  # mlp | neural_ode | hnn | pinn
hidden_dims = "124 124 124 124"  # Space-separated
activation = "tanh"  # tanh | relu | gelu | silu | softplus
use_batch_norm = False
dropout_rate = 0.0
final_activation = None  # None | tanh | sigmoid
input_dim = 5
output_dim = 2

# Training
lr = 0.0005
batch_size = 128
epochs = 500
optimizer = "adam"  # adam | adamw | sgd
weight_decay = 0.0
grad_clip = None
scheduler = None  # None | cosine | step

# PyTorch Optimizations
use_compile = False
compile_mode = "default"  # default | reduce-overhead | max-autotune
mixed_precision = False
gradient_accumulation_steps = 1

# Regularization
l1_lambda = 0.0
l2_lambda = 0.001

# Physics / PINN
data_loss_ratio = 0.99
residual_type = "lagrangian"  # eom | lagrangian | hamiltonian

# Logging
log_interval = 10
print_interval = 50
save_checkpoints = True
checkpoint_interval = 50
test_interval = 100

# Early Stopping
early_stopping_patience = 100  # None to disable

# Physical Parameters
m1 = 1.0
m2 = 1.0
l1 = 1.0
l2 = 1.0
g = 9.81

# Build command arguments
args_list = [
    f"--seed {seed}",
    f"--hidden_dims {hidden_dims}",
    f"--input_dim {input_dim}",
    f"--output_dim {output_dim}",
    f"--residual_type {residual_type}",
    f"--epochs {epochs}",
    f"--lr {lr}",
    f"--batch_size {batch_size}",
    f"--data_dir {data_dir}",
    f"--val_split {val_split}",
    f"--test_split {test_split}",
    f"--model {model}",
    f"--activation {activation}",
    f"--dropout_rate {dropout_rate}",
    f"--optimizer {optimizer}",
    f"--weight_decay {weight_decay}",
    f"--compile_mode {compile_mode}",
    f"--gradient_accumulation_steps {gradient_accumulation_steps}",
    f"--l1_lambda {l1_lambda}",
    f"--l2_lambda {l2_lambda}",
    f"--data_loss_ratio {data_loss_ratio}",
    f"--log_interval {log_interval}",
    f"--print_interval {print_interval}",
    f"--test_interval {test_interval}",
    f"--checkpoint_interval {checkpoint_interval}",
    f"--m1 {m1}",
    f"--m2 {m2}",
    f"--l1 {l1}",
    f"--l2 {l2}",
    f"--g {g}"
]

# Add optional flags
if run_name:
    args_list.append(f"--run_name {run_name}")
if checkpoint_path:
    args_list.append(f"--checkpoint_path {checkpoint_path}")
if use_compile:
    args_list.append("--use_compile")
if normalize_time:
    args_list.append("--normalize_time")
if normalize_state:
    args_list.append("--normalize_state")
if use_batch_norm:
    args_list.append("--use_batch_norm")
if final_activation:
    args_list.append(f"--final_activation {final_activation}")
if grad_clip:
    args_list.append(f"--grad_clip {grad_clip}")
if scheduler:
    args_list.append(f"--scheduler {scheduler}")
if mixed_precision:
    args_list.append("--mixed_precision")
if save_checkpoints:
    args_list.append("--save_checkpoints")
if early_stopping_patience:
    args_list.append(f"--early_stopping_patience {early_stopping_patience}")

args = " ".join(args_list)

print(f"Training Configuration:")
print(f"  Seed: {seed}")
print(f"  Model: {model}, Hidden: {hidden_dims}, Activation: {activation}")
print(f"  Optimizer: {optimizer}, LR: {lr}, Weight Decay: {weight_decay}")
print(f"  Epochs: {epochs}, Early Stop: {early_stopping_patience if early_stopping_patience else 'disabled'}")
print(f"  Batch size: {batch_size}")
print(f"  Data loss ratio = {data_loss_ratio}, type={residual_type}")
print(f"  Checkpoint: {checkpoint_path if checkpoint_path else 'None (training from scratch)'}")
print(f"\nCommand arguments generated with {len(args_list)} parameters")


Training Configuration:
  Seed: 42
  Model: pinn, Hidden: 64 64 64 64, Activation: tanh
  Optimizer: adam, LR: 0.0005, Weight Decay: 0.0
  Epochs: 500, Early Stop: 100
  Batch sizes: data=128, colloc=1024
  Data loss ratio = 0.99, type=lagrangian
  Checkpoint: None (training from scratch)

Command arguments generated with 42 parameters


In [ ]:
# Launch distributed training with Accelerate
!accelerate launch --num_processes=2 {project_root}/scripts/train.py {args}

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
t_start = 0
t_end = 1
num_points = 1000
theta1 = 0.5
theta2 = 0.2
omega1 = 0
omega2 = 0
checkpoint = f"{base_path}/runs/run_20260112_152535/checkpoints/best_model.pth"
output_dir = f"{base_path}/runs/test_inference"

arg_list = [
    f"{checkpoint}",
    f"--output-dir {output_dir}",
    f"--t-start {t_start}",
    f"--t-end {t_end}",
    f"--omega1 {omega1}",
    f"--omega2 {omega2}",
    f"--theta1 {theta1}",
    f"--theta2 {theta2}",
]
args = " ".join(arg_list)
!python {project_root}/scripts/inference.py {args}

Loading model from: /kaggle/working/runs/run_20260112_152535/checkpoints/best_model.pth
Normalization settings:
  Time: True | Range: [0.000, 3.000]
  Angles: True | Range: [-3.142, 3.142]

Initial state: theta1=0.473, theta2=-0.635, omega1=-0.425, omega2=-0.124
Time span: (0.0, 1.0), Points: 1000

Simulating with PINN...
Saved trajectory: /kaggle/working/runs/test_inference/trajectory_pinn.npz
Saved parameters: /kaggle/working/runs/test_inference/trajectory_pinn_parameters.json

Computing ground truth...
Deriving equations symbolically (this may take a moment)...
Symbolic derivation complete!
Saved trajectory: /kaggle/working/runs/test_inference/trajectory_true.npz
Saved parameters: /kaggle/working/runs/test_inference/trajectory_true_parameters.json

Generating comparison plots...
Comparison plot saved to: /kaggle/working/runs/test_inference/comparison_plot.png

Computing metrics...
Saved metrics: /kaggle/working/runs/test_inference/metrics.json

INFERENCE RESULTS
Position RMSE:  0.37

In [72]:
!zip -r /kaggle/working/runs.zip runs

updating: runs/ (stored 0%)
updating: runs/test_inference/ (stored 0%)
updating: runs/test_inference/trajectory_true.npz (deflated 11%)
updating: runs/test_inference/trajectory_pinn.npz (deflated 24%)
updating: runs/test_inference/trajectory_pinn_parameters.json (deflated 48%)
updating: runs/test_inference/trajectory_true_parameters.json (deflated 48%)
updating: runs/test_inference/metrics.json (deflated 48%)
updating: runs/test_inference/comparison_plot.png (deflated 9%)
  adding: runs/run_20260112_152535/ (stored 0%)
  adding: runs/run_20260112_152535/metrics.csv (deflated 51%)
  adding: runs/run_20260112_152535/test_results.txt (deflated 66%)
  adding: runs/run_20260112_152535/checkpoints/ (stored 0%)
  adding: runs/run_20260112_152535/checkpoints/best_model.pth (deflated 11%)
  adding: runs/run_20260112_152535/events.out.tfevents.1768231537.2429de81f59c.221803.0 (deflated 67%)


In [65]:
!rm -rf runs